In [80]:
import numpy as np
import pandas as pd
from typing import Optional, Tuple, Dict, List
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm

In [26]:
def convert_data_into_input_for_train(train):
    trainTimes = train["timestamp"]



    numberOfData = train["timestamp"].size
    # print(numberOfData)
    # raise Exception("oj")
    
    
    # print(train.shape)
    zdej = pd.DataFrame()
    prej = pd.DataFrame()
    prejPrej = pd.DataFrame()
  
    dictZdej = {}
    dictPrej = {}
    dictPrejPrej = {}
    for key in train :
        dictZdej[key] = []
        dictPrej[key] = []
        dictPrejPrej[key] = []
    hour = 3600
    timestamp1 = []
    # timestamp2 = []

    for i, date in enumerate(trainTimes):

        enoPrej = date.hour-1
        dvePrej = date.hour-2
        if(enoPrej < 0):
            enoPrej = 23
        if(dvePrej < 0):
            dvePrej = 23

        timestampEnoPrej = date.replace(hour = enoPrej)
        timestampDvePrej = date.replace(hour = dvePrej)

        closest = np.argmin(np.abs(trainTimes - date))
        closest1 = np.argmin(np.abs(trainTimes - timestampEnoPrej))
        closest2 = np.argmin(np.abs(trainTimes - timestampDvePrej))
        timediff2 = date - trainTimes[closest2]
        timediff1 = date - trainTimes[closest1]


        timediff2 = timediff2.total_seconds()
        timediff1 = timediff1.total_seconds()

        if(timediff2 > 2*hour-400 and timediff2 < hour*2 +400):
            # 2h obstaja
             if(timediff1 >hour-400 and timediff1< hour+400):
                #  1h obstaja
                timestamp1.append(date)
                for key in train:
                    dictPrejPrej[key].append(train[key][closest2])
                    dictPrej[key].append(train[key][closest1])
                    dictZdej[key].append(train[key][closest])

    zdej["timestamp"] = timestamp1
    prej["timestamp"] = timestamp1
    prejPrej["timestamp"] = timestamp1

    for key in train:
        if key =="timestamp":
            continue
        # print(dictt1[key])
        # raise "ajj"
        zdej[key] = dictZdej[key]
        prej[key] = dictPrej[key]
        prejPrej[key] = dictPrejPrej[key]


    zdej.to_csv("TrainZdej.csv", sep=",", index=False)
    prej.to_csv("TrainPrej.csv", sep=",", index=False)
    prejPrej.to_csv("TrainPrejPrej.csv", sep=",", index=False)



In [100]:
def magic_train(zdej,prej,prejPrej,postaja):
    
    numberOfData = zdej["timestamp"].size
    
    dveNazaj = np.zeros([numberOfData,1])
    enoNazaj = np.zeros([numberOfData,1])

    uraDneva = np.zeros([numberOfData,24])
    weekDay = np.zeros([numberOfData,7])
    minute = np.zeros([numberOfData,1])
    pocitnice = np.zeros([numberOfData,1])

    
    for i, date in enumerate(zdej["timestamp"]):
        closest = np.argmin(np.abs(prej["timestamp"] - date))
        
        dveNazaj[i] = prej[postaja][closest]
        enoNazaj[i] = prejPrej[postaja][closest]
        # if(closest ==i):
        #     print("dela")
        # print(prej[postaja][i])

        uraDneva[i,date.hour] = 1
        weekDay[i,date.weekday()] = 1
        minute[i] = date.minute
        if( date.month <9):
            pocitnice[i] = 1



    X = np.concatenate((dveNazaj , enoNazaj, uraDneva,weekDay,minute,pocitnice), axis=1)
    return X

In [94]:
if __name__ == "__main__":
    train = pd.read_csv("bicikelj_train.csv")
    train["timestamp"] = [pd.to_datetime(ts).tz_localize(None) for ts in train["timestamp"].values]

    test = pd.read_csv("bicikelj_test.csv")
    test["timestamp"] = [pd.to_datetime(ts).tz_localize(None) for ts in test["timestamp"].values]

    mojDataResult = pd.DataFrame()
    mojDataResult["timestamp"] = test["timestamp"]
    

In [ ]:
# Only for creating zdej prej prejPrej files, run first time
convert_data_into_input_for_train(train)
print("HELLO")

In [95]:
zdej = pd.read_csv("TrainZdej.csv")
zdej["timestamp"] = [pd.to_datetime(ts).tz_localize(None) for ts in zdej["timestamp"].values]

prej = pd.read_csv("TrainPrej.csv")
prej["timestamp"] = [pd.to_datetime(ts).tz_localize(None) for ts in prej["timestamp"].values]

prejPrej = pd.read_csv("TrainPrejPrej.csv")
prejPrej["timestamp"] = [pd.to_datetime(ts).tz_localize(None) for ts in prejPrej["timestamp"].values]


In [101]:
for i, postaja in tqdm(enumerate(zdej)):
    if(postaja == "timestamp"):
        continue
    ridge = Ridge( alpha=1e-05)
    X = magic_train(zdej = zdej,prej=prej,prejPrej= prejPrej,postaja=postaja)
    # print(X)
    # 
    y = zdej[postaja] # se np array nardit?
    y = np.array(y)
    # print(y)
    Xtest = magic_train(zdej = test,prej=zdej,prejPrej= prej,postaja=postaja)
    min = np.min(y)
    max = np.max(y)
    ridge.fit(X,y)
    result = ridge.predict(Xtest)

    for i in range(len(result)):
            if(result[i] <min ):
                result[i]=min
            if(result[i] > max):
                result[i] = max
    result = result.round(0)

    mojDataResult[postaja] = np.array(result)
mojDataResult.to_csv("newEra.csv", sep=",", index=False)
    

84it [02:54,  2.07s/it]


In [ ]:
print("HELLO")